In [1]:
import numpy as np
from scipy.integrate import solve_ivp
from matplotlib.backends.backend_tkagg import (
    FigureCanvasTkAgg, NavigationToolbar2Tk)
from matplotlib.backend_bases import key_press_handler
from matplotlib.figure import Figure
import tkinter as tk

window = tk.Tk()
window.title('Preračun povesa')
window.geometry('600x1050')

### FUNKCIJE ###
def fun_y(z, y, M, F_y, L, E, I):
    '''
    Pripravljena funkcija za izračun povesa z uporabo reševanja diderencialnih enačb.
    
    z     - neodvisna spremenljivka (dolžina konzole)
    y     - odvisna spremenljivka (poves)
    *args - podamo ostale parametre enačbe
    '''
    ddy =  -(M + F_y*(L-z))/(E*I)
    return y[1], ddy

def shrani_in_izracunaj():
    '''
    Funkcija, ki zbere podatke in naredi preračun.
    '''
    ## pridobimo numerične konstante ##
    M_num=float(entry1.get())#Nmm
    M = M_num*1000
    
    F_num = float(entry2.get())#N
    F_y=np.cos(np.pi/6)*F_num
    
    L_num=float(entry3.get())#mm
    L = L_num*1000
    
    E=float(entry4.get())#MPa
    
    I=float(entry5.get())
    
    args=(M, F_y, L, E, I)
    ## neodvisna spremenljivka ##
    z = np.linspace(0, L, 100) #mm
    ## začetne vrednosti iz robnih pogojev ##
    y0=0
    dy0=0
    zac_vr = [y0,dy0]
    
    resitev = solve_ivp(lambda z, y: fun_y(z, y, *args), #funkcijo preoblikujemo tako, da sprejme dve spremenljivke
                        t_span=(z[0], z[-1]),            #interval reševanja
                        y0=zac_vr,                       #začetni pogoji
                        method='RK45',                   #metoda reševanja (Runge-Kutta četrtega reda) 
                        t_eval=z)                        #število vrnjenih vrednosti
    
    y, dy = resitev.y
    max_poves= str(abs(round(y[-1], ndigits=5)))
    
    fig = Figure(figsize=(5, 4), dpi=100)
    t = np.arange(0, 3, .01)
    fig.add_subplot(111).plot(z, y)

    canvas = FigureCanvasTkAgg(fig, master=window)  # A tk.DrawingArea.
    canvas.draw()
    canvas.get_tk_widget().grid(column=0, row=13, columnspan=6)
    
    return 'Maximalni poves na koncu nosilca je '+max_poves+' mm'
    

def prikaz_rezultata():
    '''
    Prikaže izračun.
    '''
    rezultat = shrani_in_izracunaj()
    rezultat_prikaz = tk.Text(master=window, height=1, width=60)
    rezultat_prikaz.grid(column=0, row=11, columnspan=4)
    
    rezultat_prikaz.insert(tk.END, rezultat)
    

### ZNAMKE IN SLIKE ###
title = tk.Label(text='Določite poves nosilca glede na vpisano obremenitev!', font=('Calibri', 20))
title.grid(column=0,row=0, columnspan=4)

photo = tk.PhotoImage(file='Slike/3D_konzola_GUI.png')
slika = tk.Label(image=photo)
slika.grid(column=0, row=2, columnspan=4)

label1 = tk.Label(text='Za primer preproste konzole lahko določite njen poves. Uporabi se rešitev Eulerjeve diferencialne enačbe.')
label1.grid(column=0,row=1, columnspan=4)

label2 = tk.Label(text='_________________________________________________________________________________________________________')
label2.grid(column=0, row=3, columnspan=4)

label3 = tk.Label(text='Vnesite vaše parametre za izračun ( Vnesite prave števične vrednosti! )', font=('Calibri', 14))
label3.grid(column=0, row=4, columnspan=4)

label4 = tk.Label(text='M [Nm]:')
label4.grid(column=0, row=5, sticky=tk.E)

label4 = tk.Label(text='F [N]:')
label4.grid(column=0, row=6, sticky=tk.E)

label4 = tk.Label(text='L [m]:')
label4.grid(column=2, row=5, sticky=tk.E)

label4 = tk.Label(text='E [MPa]:')
label4.grid(column=2, row=6, sticky=tk.E)

label4 = tk.Label(text='I [mm^4]:')
label4.grid(column=2, row=7, sticky=tk.E)

label5 = tk.Label(text='_________________________________________________________________________________________________________')
label5.grid(column=0, row=8, columnspan=6)

label6 = tk.Label(text='          ')
label6.grid(column=0, row=10, columnspan=6)

label7 = tk.Label(text=' ')
label7.grid(column=0, row=12, columnspan=6)

#### ENTRY FIELDS ###
entry1 = tk.Entry()
entry1.grid(column=1, row=5, sticky=tk.W)

entry2 = tk.Entry()
entry2.grid(column=1, row=6, sticky=tk.W)

entry3 = tk.Entry()
entry3.grid(column=3, row=5, sticky=tk.W)

entry4 = tk.Entry()
entry4.grid(column=3, row=6, sticky=tk.W)

entry5 = tk.Entry()
entry5.grid(column=3, row=7, sticky=tk.W)

### BUTTONS ###
button1 = tk.Button(text='Izračun', bg='green', command=prikaz_rezultata)
button1.grid(column=0, row=9, columnspan=4)


# Prikaže okence
window.mainloop()